In [1]:
import pandas as pd
import data_utils as data

source = data.Data.BF_1.value[0]

print(source['path'])

df_flow, df_cpu, df_memory, df_bw_in, df_bw_out, df_pkts_in, df_pkts_out, df_iops_rw, df_iops, df_tp_rw, df_tp = data.get_data(source['path'])

save_directory = source['path'].replace('data', 'output' + '\\')
start_date = source['start_time']
end_date = source['end_time']

print(df_flow.shape)
df_flow.head()
print(df_flow.nunique())

.\data\BruteForce\21-03-2024-3h
(221552, 80)
class                    2
dst_port              8153
protocol                 2
timestamp            20187
flow_duration       101339
                     ...  
cwe_flag_count           1
subflow_fwd_pkts       231
subflow_bwd_pkts       233
subflow_fwd_byts      5865
subflow_bwd_byts      6176
Length: 80, dtype: int64


Setup

Time filtering

In [ ]:
df_flow['timestamp'] = pd.to_datetime(df_flow['timestamp'])

print(df_flow.shape)

# Filter rows between start_date and end_date
df_flow = df_flow[(df_flow['timestamp'] >= start_date) & (df_flow['timestamp'] <= end_date)]

print(df_flow.shape)

# Replace class values ([0] -> 0 and [1] -> 1)
df_flow['class'] = df_flow['class'].replace({'[0]': 0, '[1]': 1})
# filtered_df.head()
df_flow[df_flow['class']==1]

In [ ]:
# # Only for brute force data
# df_sorted = df_flow.sort_values(by='timestamp')
# split_index = len(df_sorted) // 2
# df_second_half = df_sorted.iloc[split_index:]
# df_second_half.reset_index(drop=True, inplace=True)
# df_flow = df_second_half

In [ ]:
# Save flow data
df_flow.to_csv(save_directory + "flows.csv", index=False)

Column filtering

In [ ]:
# df_flow = df_flow.get(["timestamp", "class", "flow_duration", "flow_byts_s", "flow_pkts_s", "tot_fwd_pkts", "tot_bwd_pkts", "totlen_fwd_pkts", "totlen_bwd_pkts"])
# print(df_flow.shape)
# df_flow.head()

Prepare resource data

In [ ]:
df_cpu = pd.read_csv(CPU_PATH)
df_cpu = df_cpu.rename(columns={'Time': 'timestamp'})
df_cpu = df_cpu.drop("Value 1", axis=1)
# Merge CPU values by sum
df_cpu["cpu_total"] = df_cpu.drop("timestamp", axis=1).sum(axis=1)
df_cpu.head()
# Save CPU data
df_cpu.to_csv(save_directory + "cpu.csv", index=False)

In [ ]:
df_bw_in = pd.read_csv(BW_IN_PATH)
df_bw_in = df_bw_in.rename(columns={'Time': 'timestamp'})
df_bw_in.head()
df_bw_in.to_csv(save_directory + "bw_in.csv", index=False)

In [ ]:
df_bw_out = pd.read_csv(BW_OUT_PATH)
df_bw_out = df_bw_out.rename(columns={'Time': 'timestamp'})
df_bw_out.head()
df_bw_out.to_csv(save_directory + "bw_in.csv", index=False)

Convert timestamp columns to datetime objects

In [ ]:
df_cpu['timestamp'] = pd.to_datetime(df_cpu['timestamp'])
df_flow['timestamp'] = pd.to_datetime(df_flow['timestamp'])

Data mapping (method 1)

In [ ]:
df_flow1 = df_flow

# Create a new column in the network dataframe to store associated cpu records
df_flow1['cpu'] = None

# Loop over each flow row
for index, flow_row in df_flow1.iterrows():
    # Find the nearest cpu record according to time
    nearest_cpu_record = df_cpu.iloc[(df_cpu['timestamp'] - flow_row['timestamp']).abs().argsort()[0]]

    # Assign the nearest cpu record to the current flow row
    df_flow1.at[index, 'cpu'] = nearest_cpu_record['cpu_total']

# Convert all timestamps to total seconds from the first timestamp
df_flow1 = df_flow1.sort_values(by='timestamp')
first_timestamp = df_flow1['timestamp'].iloc[0]
df_flow1['timestamp'] = (df_flow1['timestamp'] - first_timestamp).dt.total_seconds()

# Save flow data
df_flow1.to_csv(save_directory + "data1.csv", index=False)

# Print the updated network dataframe with associated cpu records
print(df_flow1)

Data mapping (method 2)

In [ ]:
df_cpu1 = df_cpu

# Function to aggregate flows for each CPU record
def aggregate_flows(cpu_row, network_df):
    # Get the current and next CPU timestamps
    current_timestamp = cpu_row['timestamp']
    next_timestamp = df_cpu1.loc[cpu_row.name + 1, 'timestamp'] if cpu_row.name + 1 < len(df_cpu) else pd.Timestamp.max

    # Filter flows that fall within the time interval
    flows_in_interval = network_df[(network_df['timestamp'] >= current_timestamp) & (network_df['timestamp'] < next_timestamp)]

    # Aggregate flow values
    row = flows_in_interval.drop(['timestamp', 'protocol', 'dst_port'], axis=1).sum()

     # Add a column for the number of aggregated flows
    row['num_flows'] = len(flows_in_interval)

    if len(flows_in_interval) != 0:
        features = row.index[(row.index != 'num_flows') & (row.index != 'class')]
        row[features] = row[features] / row['num_flows']

    return row

# Loop over each CPU row
for index, cpu_row in df_cpu1.iterrows():
    # Aggregate flows for the current CPU row
    aggregated_values = aggregate_flows(cpu_row, df_flow)

    # Add aggregated flow values to the CPU row
    df_cpu1.loc[index, aggregated_values.index] = aggregated_values.values

# Fill NaN values with 0
df_cpu1.fillna(0, inplace=True)

# Convert all timestamps to total seconds from the first timestamp
df_cpu1 = df_cpu1.sort_values(by='timestamp')
first_timestamp = df_cpu1['timestamp'].iloc[0]
df_cpu1['timestamp'] = (df_cpu1['timestamp'] - first_timestamp).dt.total_seconds()

# Save flow data
df_cpu1.to_csv(save_directory + "data2.csv", index=False)

# Print the updated CPU dataframe
print(df_cpu1)